In [1]:
!pip install keras-tuner

     |████████████████████████████████| 97 kB 2.3 MB/s 


In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

def build_model(hp):
  input_node = keras.Input(shape=(20,))
  units = hp.Int('units', min_value=32, max_value=512, step=32)
  output_node = layers.Dense(units=units, activation='relu')(input_node)
  output_node = layers.Dense(units=units, activation='relu')(output_node)
  output_node = layers.Dense(units=1, activation='sigmoid')(output_node)
  model = keras.Model(input_node, output_node)

  optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
  model.compile(
    optimizer=optimizer,
    loss='mse',
    metrics=['mae'])
  return model

In [3]:
from keras_tuner import RandomSearch

tuner = RandomSearch(
    build_model,
    objective='val_mae',
    max_trials=5,
    executions_per_trial=3,
    directory='my_dir',
    project_name='helloworld')

In [4]:
tuner.search_space_summary()

Search space summary
Default search space size: 1
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}


In [5]:
import numpy as np
x_train = np.random.rand(100, 20)
y_train = np.random.rand(100, 1)
x_val = np.random.rand(20, 20)
y_val = np.random.rand(20, 1)

tuner.search(x_train, y_train, epochs=1, validation_data=(x_val, y_val))

Trial 5 Complete [00h 00m 02s]
val_mae: 0.22459365924199423

Best val_mae So Far: 0.21783710519472757
Total elapsed time: 00h 00m 24s
INFO:tensorflow:Oracle triggered exit


In [6]:
tuner.results_summary(5)

Results summary
Results in my_dir/helloworld
Showing 5 best trials
Objective(name='val_mae', direction='min')
Trial summary
Hyperparameters:
units: 224
Score: 0.21783710519472757
Trial summary
Hyperparameters:
units: 416
Score: 0.21908955772717795
Trial summary
Hyperparameters:
units: 192
Score: 0.22459365924199423
Trial summary
Hyperparameters:
units: 352
Score: 0.22488305469353995
Trial summary
Hyperparameters:
units: 448
Score: 0.22935916483402252


In [7]:
from tensorflow import keras
best_models = tuner.get_best_models(num_models=2)
best_model = best_models[0]
best_model.save('path_to_best_model')
best_model = keras.models.load_model('path_to_best_model')
print(best_model.predict(x_val))
best_model.summary()

INFO:tensorflow:Assets written to: path_to_best_model/assets
[[0.598885  ]
 [0.5619056 ]
 [0.5612072 ]
 [0.55784094]
 [0.5542791 ]
 [0.5752317 ]
 [0.568797  ]
 [0.53858554]
 [0.5366706 ]
 [0.5515216 ]
 [0.529739  ]
 [0.565692  ]
 [0.5646275 ]
 [0.58155847]
 [0.55058855]
 [0.55425227]
 [0.5594124 ]
 [0.542532  ]
 [0.5591536 ]
 [0.5534017 ]]
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 20)]              0         
_________________________________________________________________
dense (Dense)                (None, 224)               4704      
_________________________________________________________________
dense_1 (Dense)              (None, 224)               50400     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 225       
Total params: 55,329
Trainable params: 55,329
Non

In [8]:
def build_model(hp):
  input_node = keras.Input(shape=(20,))
  units = hp.Int('units', min_value=32, max_value=512, step=32)
  output_node = layers.Dense(units=units, activation='relu')(input_node)
  output_node = layers.Dense(units=units, activation='relu')(output_node)
  output_node = layers.Dense(units=1, activation='sigmoid')(output_node)
  model = keras.Model(input_node, output_node)
  optimizer_name = hp.Choice('optimizer', ['adam', 'adadelta'])
  learning_rate = hp.Float('learning_rate', min_value=1e-5, max_value=0.1,
                           sampling='log')
  if optimizer_name == 'adam':
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
  else:
    optimizer = tf.keras.optimizers.Adadelta(learning_rate=learning_rate)
  model.compile(
    optimizer=optimizer,
    loss='mse',
    metrics=['mae'])
  return model

tuner = RandomSearch(
    build_model,
    objective="val_mae",
    max_trials=5,
    executions_per_trial=3,
    directory='my_dir',
    project_name='helloworld')

INFO:tensorflow:Reloading Oracle from existing project my_dir/helloworld/oracle.json
INFO:tensorflow:Reloading Tuner from my_dir/helloworld/tuner0.json


In [9]:
from tensorflow.keras.layers.experimental.preprocessing import Normalization

layer = Normalization(input_shape=(20,))
layer.adapt(x_train)

model = tf.keras.Sequential([layer, tf.keras.layers.Dense(1)])
model.compile(optimizer='adam', loss='mse')
model.fit(x_train, y_train)

4/4 [==============================] - 0s 3ms/step - loss: 1.8392


In [10]:
from keras_tuner import HyperModel

class Regressor(HyperModel): 
  def __init__(self, data):
    self.data = data

  def build(self, hp):
    model = tf.keras.Sequential()
    if hp.Boolean('normalize'):
        layer = Normalization(input_shape=(20,))
        layer.adapt(self.data)
        model.add(layer)
    model.add(tf.keras.layers.Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

hypermodel = Regressor(x_train)

In [11]:
tuner = RandomSearch(hypermodel, objective='val_loss', max_trials=2)
tuner.search(x_train, y_train, validation_data=(x_val, y_val))

Trial 2 Complete [00h 00m 00s]
val_loss: 2.238662004470825

Best val_loss So Far: 0.217946857213974
Total elapsed time: 00h 00m 01s
INFO:tensorflow:Oracle triggered exit
